In [1]:
!pip install streamlit
!pip install tiktoken
!pip install openai
!pip install langchain-openai
!pip install nltk






In [1]:
import os
import streamlit as st
import pickle
import time
import langchain
import nltk
import os
from langchain_openai import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [2]:
#load openAI api key
os.environ['OPENAI_API_KEY'] = 'sk-proj-Px3RAzJ37V9sYPFWGwaoT3BlbkFJAOcVggMjHmfhskjdhfjshfjk'

In [3]:
# Initialise LLM with required params
llm = OpenAI(temperature=0.9, max_tokens=500) 

loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

2

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)
len(docs)

16

In [5]:
docs[9]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nFREE Credit Score₹100 Cash Reward\n\nFixed Deposits\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nFREE Credit Score₹100 Cash Reward\n\nFixed Deposits\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_BUSINESS_AS/MC_ENG_ROS_NWS_BUS_AS_ATF_728\n\nGo PRO @₹99 PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHomeNewsBusinessTata Motors launches Punch iCNG, price starts at Rs 7.1 lakh\n\nTrending Topics\n\nBajaj Housi

In [6]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

/var/folders/3r/hn_7sgwj3vsfzw1_c3ss2bm80000gn/T/ipykernel_4869/2920682162.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()


In [7]:
# Storing vector index create in local
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import OpenAIEmbeddings

vectorindex_openai.save_local("vectorindex_openai")
x = FAISS.load_local("vectorindex_openai", embeddings, allow_dangerous_deserialization=True)


In [8]:
# Use `as_retriever()` method on the correct object
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex_openai.as_retriever())
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x13956d010>, async_client=<openai.resources.completions.AsyncCompletions object at 0x13aef6060>, temperature=0.9, max_tokens=500, model_kwargs={}, openai_api_key=SecretStr('**********'), logit_bias={}), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input

In [9]:
query = "what is the price of Tiago iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

/var/folders/3r/hn_7sgwj3vsfzw1_c3ss2bm80000gn/T/ipykernel_4869/3133678122.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chain({"question": query}, return_only_outputs=True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}


Error in ConsoleCallbackHandler.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Parent run a978811c-bf41-40be-93f2-53e25d09daf9 not found for run 690743c7-e669-404e-bbd5-aecb96b95974. Treating as a root run.
Parent run a978811c-bf41-40be-93f2-53e25d09daf9 not found for run 70c2a625-8c73-4d66-9b76-ed96f381f165. Treating as a root run.
Parent run a978811c-bf41-40be-93f2-53e25d09daf9 not found for run 1146272a-f61d-4ccf-b995-59f4f2f246e1. Treating as a root run.
Parent run a978811c-bf41-40be-93f2-53e25d09daf9 not found for run 3deaf657-b26f-4bbc-bada-fd7c52012f57. Treating as a root run.


[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[llm/start] [llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\nThe company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nTags: #Business #Companies\n\nfirst published: Aug 4, 2023 02:17 pm\n\nTop Trends\n\nAngel TaxWiproBudget newsNew Income tax slabIPO News\n\nAdvertisement\n\nRemove Ad\n\nAdvertisement\n\nRemove Ad\n\nAdvertiseme

Error in ConsoleCallbackHandler.on_chain_end callback: TracerException('No indexed run ID a978811c-bf41-40be-93f2-53e25d09daf9.')


[llm/end] [llm:OpenAI] [1.78s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "total_tokens": 1308,
      "prompt_tokens": 1145,
      "completion_tokens": 163
    },
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null,
  "type": "LLMResult"
}
[llm/end] [llm:OpenAI] [1.78s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The Punch iCNG is equipped with the company's proprietary twin-cylinder technology with enhanced safety features like a micro-switch to keep the car switched off at the time of refuelling and thermal incident protection that cuts off CNG supply to the engine and releases gas into the atmosphere, Tata Motors said in a statement. Tata Mot

{'answer': ' The price of the Tiago iCNG is between Rs 6.55 lakh and Rs 8.1 lakh.\n',
 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}